# Download code and Data

## This will take a few minutes...

In [123]:
!wget https://github.com/jrgillick/imagined-lyrics/archive/master.zip
!unzip master.zip
!rm master.zip
!mv imagined-lyrics-master/ imagined-lyrics



!pip install praatio
!pip install tensorboardX
!pip install pytorch_pretrained_bert
import sys, random, time, math, copy, shutil
sys.path.append('imagined-lyrics')
sys.path.append('imagined-lyrics/utils')
import models
import audio_utils, dataset_utils, data_loaders, torch_utils, text_utils
import os, pandas as pd, librosa, numpy as np
from IPython.display import Audio
from tqdm import tqdm_notebook as tqdm
import torch
from torch.utils import data
from torch.autograd import Variable
from functools import partial
from models import *
import itertools
from keras.preprocessing.sequence import pad_sequences as keras_pad_seqs
from google.colab import files



import gdown

# Download phoneme model
!mkdir phoneme_recognition_model
url = 'https://drive.google.com/uc?id=1T1W2t-T5YNTBJPoL09wd4W48HiZ25LcW'
output = 'phoneme_recognition_model/best.pth.tar'
gdown.download(url, output, quiet=False)

# Download lyrics decoder model
!mkdir phonemes_to_words_model
url = 'https://drive.google.com/uc?id=1Nu8-4J8iIPRS8-9b1q--FYI8rOykQUuW'
output = 'phonemes_to_words_model/best.pth.tar'
gdown.download(url, output, quiet=False)

# Download GloVe 100dim word vectors
url = 'https://drive.google.com/uc?id=1xgb9mO2mnLtWNiyurPYUUefSVJ140MZL'
output = 'glove.6B.100d.txt'
gdown.download(url, output, quiet=False)




# Download GPT model for re-scoring outputs
import math
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel, OpenAIGPTLMHeadModel
# Load pre-trained model (weights)
gpt_model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
gpt_model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')



phoneme_output_vocab = text_utils.make_vocab(
    filepaths=['imagined-lyrics/data/librispeech/libri_speech_phonemes.vocab'],
    token_fn=lambda x: open(x).read().split('\n'),
    include_pad_symbol=True, include_start_symbol=True,
    include_end_symbol=True, include_oov_symbol=True,
    standard_special_symbols=True
)

reverse_phoneme_output_vocab = text_utils.make_reverse_vocab(phoneme_output_vocab)

INPUT_DIM = 37
OUTPUT_DIM = len(phoneme_output_vocab)
HID_DIM = 256

max_label_len =13
max_seq_len=180

print("Initializing model...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

enc = Encoder(INPUT_DIM, HID_DIM, dropout=0.)
dec = Decoder(OUTPUT_DIM, HID_DIM, dropout=0., device=device)

phoneme_model = Seq2Seq(enc, dec, device, max_label_len)

phoneme_model_ckpt = 'phoneme_recognition_model/best.pth.tar'
checkpoint = torch_utils.load_checkpoint(phoneme_model_ckpt, phoneme_model)
phoneme_model.set_device(device)
_ = phoneme_model.eval()



def separate_vox(y, sr):
    S_full, phase = librosa.magphase(librosa.stft(y))
    S_filter = librosa.decompose.nn_filter(S_full,
        aggregate=np.median, metric='cosine',
        width=int(librosa.time_to_frames(2, sr=sr)))
    
    S_filter = np.minimum(S_full, S_filter)
        
    margin_i, margin_v = 2, 10
    power = 2
    
    mask_i = librosa.util.softmask(S_filter,
                                       margin_i * (S_full - S_filter),
                                       power=power)
    
    mask_v = librosa.util.softmask(S_full - S_filter,
                                       margin_v * S_filter,
                                       power=power)
    
    # Once we have the masks, simply multiply them with the input spectrum
    # to separate the components
    
    S_foreground = mask_v * S_full
    S_background = mask_i * S_full

    foreground_audio = librosa.istft(S_foreground)
    background_audio = librosa.istft(S_background)

    return foreground_audio, background_audio

def load_audio(f, offset, duration, sr=16000, mono=True):
    lr, sr = librosa.load(f, sr=sr,offset=offset, duration=duration, mono=mono)
    return lr, sr

def combine_window_outputs(window_outputs):
    l = []
    for o in window_outputs:
        l += [e for e in o if e!= 'pause']
    return l



glove = text_utils.glove2dict("glove.6B.100d.txt")

d = dataset_utils.read_cmu_pronunciation_dict('imagined-lyrics/data/librispeech/english.dict')
input_vocab = text_utils.make_vocab(
    filepaths=['imagined-lyrics/data/librispeech/libri_speech_phonemes.vocab'],
    token_fn=lambda x: open(x).read().split('\n'),
    include_pad_symbol=True, include_start_symbol=True,
    include_end_symbol=True, include_oov_symbol=True,
    standard_special_symbols=True
)

word_output_vocab = text_utils.make_vocab(filepaths=['imagined-lyrics/data/metrolyrics/vocabs/words.txt'],
                token_fn=lambda f: list(text_utils.glove2dict(f).keys()),
                include_pad_symbol=True, include_start_symbol=True,
                include_end_symbol=True, include_oov_symbol=True,
                standard_special_symbols=True)

reverse_word_output_vocab = text_utils.make_reverse_vocab(word_output_vocab)

output_embedding_matrix = text_utils.make_embedding_matrix(glove, word_output_vocab, 100)

INPUT_DIM = len(input_vocab)
OUTPUT_DIM = len(word_output_vocab)
DEC_EMB_DIM = 100
ENC_DROPOUT = 0.
DEC_DROPOUT = 0.

word_enc = Encoder(INPUT_DIM, HID_DIM, ENC_DROPOUT)
word_dec = EmbeddingDecoder(OUTPUT_DIM, HID_DIM, DEC_EMB_DIM, output_embedding_matrix, word_output_vocab, DEC_DROPOUT, device)

word_model = Seq2Seq(word_enc, word_dec, device, max_label_len)

words_model_ckpt = 'phonemes_to_words_model/best.pth.tar'
checkpoint = torch_utils.load_checkpoint(words_model_ckpt, word_model)
word_model.set_device(device)
_ = word_model.eval()



def phonemes_to_words(phoneme_seqs, model, phoneme_output_vocab, reverse_word_output_vocab,
                     dedup_outputs=True, remove_oov=True):
    if len(np.array(phoneme_seqs).shape) == 1:
        phoneme_seqs = [phoneme_seqs]
        
    phoneme_seqs = [text_utils.sequence_to_indices(phoneme_seq, phoneme_output_vocab, one_hot=True) for phoneme_seq in phoneme_seqs]
    seqs = keras_pad_seqs(phoneme_seqs, maxlen=30, dtype='float32',
            padding='pre', truncating='post', value=0)
    with torch.no_grad():
        src = torch.from_numpy(np.array(seqs)).float().permute((1,0,2)).to(model.device)
        output = word_model(src)

    # Remove batch dimension, get arxmax from one hot, and convert to numpy
    output_seqs = np.argmax(output.cpu(), axis=-1).numpy().T #np.argmax(output.cpu().numpy().squeeze(), axis=1)
    
    # Convert to readable labels with reverse vocab
    word_seqs = [text_utils.readable_outputs(output_seq, reverse_word_output_vocab) for output_seq in output_seqs]
    
    for i in range(len(word_seqs)):
        words = word_seqs[i]
        start_index = words.index(text_utils.START_SYMBOL)+1 if text_utils.START_SYMBOL in words else 0
        end_index = words.index(text_utils.END_SYMBOL) if text_utils.END_SYMBOL in words else len(words)
        words = words[start_index:end_index]
        if dedup_outputs:
            words = models.dedup_list(words)
        if remove_oov:
            words = [w for w in words if w!="###_OOV_###"]
        word_seqs[i] = words
            
            
    return word_seqs


# Function for scoring model outputs
def score(sentence):
    if type(sentence) is list:
        sentence = ' '.join(sentence)
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    loss=gpt_model(tensor_input, lm_labels=tensor_input)
    return math.exp(loss)

def imagine_phonemes(audio_file_path, phoneme_model,
                     phoneme_output_vocab, reverse_phoneme_output_vocab,
                     do_separate_vox=True, duration=None, tfm=None,
                     hop_length=1, window_length=1., offset=0.,
                     tmpfile_path = 'tmp_file',
                     num_versions=1):
    
    if duration is None:
        duration = audio_utils.get_audio_length(audio_file_path)
        
    f_in = audio_file_path
    
    if tfm is not None:
        tfm.build(f_in, tmpfile_path)
    else:
        shutil.copyfile(audio_file_path, tmpfile_path)
        
    f = tmpfile_path
        
    lr, sr = load_audio(f, 0, duration)
    
    if do_separate_vox:
        fg, bg = separate_vox(lr, sr)
        librosa.output.write_wav(tmpfile_path, fg, sr)
        
    phoneme_seqs = predict_windows(phoneme_model, f, hop_length=hop_length, window_length=window_length,
                                offset=offset, duration=duration, sr=16000,
                                reverse_output_vocab=reverse_phoneme_output_vocab,
                                num_versions=num_versions)#[1]
    
    return phoneme_seqs #return all_windows, all_combined_lines

def imagine_words(audio_file_path, phoneme_model, word_model,
                     phoneme_output_vocab, reverse_phoneme_output_vocab,
                     word_output_vocab, reverse_word_output_vocab,
                     do_separate_vox=False, duration=None, tfm=None,
                     hop_length=1, window_length=1., offset=0.,
                     dedup_outputs=True, tmpfile_path = '/home/jrgillick/tmp_sox_file.wav',
                     num_versions=1, num_tries=5):
    
    def _score(sentence):
        if type(sentence) is list:
            sentence = ' '.join(sentence)
        tokenize_input = tokenizer.tokenize(sentence)
        tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
        loss=model(tensor_input, lm_labels=tensor_input)
        return math.exp(loss)
    
    #all_windows, all_combined_lines = imagine_phonemes(
    phoneme_seqs = imagine_phonemes(
        audio_file_path, phoneme_model,
        phoneme_output_vocab, reverse_phoneme_output_vocab,
        do_separate_vox=do_separate_vox, duration=duration, tfm=tfm,
        hop_length=hop_length, window_length=window_length, offset=offset,
        tmpfile_path=tmpfile_path, num_versions=num_versions)
    
    #combined_lines = list(filter(lambda x: x!= 'pause', combined_lines))
    #windows = all_windows[0]
    #combined_lines = all_combined_lines[0]
    #phoneme_seqs = [models.dedup_list(phoneme_seq) for phoneme_seq in phoneme_seqs]
    
    word_seqs = []
    for phn in phoneme_seqs:
        phn = list(filter(lambda x: x!= 'pause', phn))
        phn = [phn for i in range(num_tries)] # try word decoder multiple times
        sents = phonemes_to_words(phn, word_model, phoneme_output_vocab,reverse_word_output_vocab, dedup_outputs=dedup_outputs)

        scores = []
        for s in sents:
            try:
                scores.append(_score(s))
            except:
                scores.append(9999999)
        if len(scores) > 0:
            best_index = np.argmin(scores)
            word_seqs.append(sents[best_index])
    return word_seqs
    #return windows, combined_lines, word_seqs#[best_index]

        

--2019-12-11 07:27:33--  https://github.com/jrgillick/imagined-lyrics/archive/master.zip
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/jrgillick/imagined-lyrics/zip/master [following]
--2019-12-11 07:27:34--  https://codeload.github.com/jrgillick/imagined-lyrics/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.253.121
Connecting to codeload.github.com (codeload.github.com)|192.30.253.121|:443... connected.
^C
unzip:  cannot find or open master.zip, master.zip.zip or master.zip.ZIP.
rm: cannot remove 'master.zip': No such file or directory
mv: cannot stat 'imagined-lyrics-master/': No such file or directory
mkdir: cannot create directory ‘phoneme_recognition_model’: File exists


Downloading...
From: https://drive.google.com/uc?id=1T1W2t-T5YNTBJPoL09wd4W48HiZ25LcW
To: /content/phoneme_recognition_model/best.pth.tar
18.1MB [00:00, 56.0MB/s]


mkdir: cannot create directory ‘phonemes_to_words_model’: File exists


Downloading...
From: https://drive.google.com/uc?id=1Nu8-4J8iIPRS8-9b1q--FYI8rOykQUuW
To: /content/phonemes_to_words_model/best.pth.tar
370MB [00:03, 93.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xgb9mO2mnLtWNiyurPYUUefSVJ140MZL
To: /content/glove.6B.100d.txt
42.5MB [00:00, 34.7MB/s]

KeyboardInterrupt: ignored

# Download a few Sample audio files to try

In [0]:
!mkdir -p sample_audio_files
names = ['bells.mp3', 'birds.mp3', 'shaker.mp3', 'talkbox.mp3', 'arp_1.wav',
         'car-horn.wav', 'drops.wav', 'skrillex.wav', 'didgeridoo.wav']
gdrive_ids = ['1GWRpdjK4IcpLDDru9LUAZpOR1oTdfxYT', '1q66IGKWUv5d1fhZ7L9JqbP1wpjvl8-yt', '1oQ9dKfUCRIHmHcrWJcCKHQMdBScrzaA3',
              '1i1fsV4y4gMGWSjJft9M7AHbTvBKgDvIV', '1aJkNKeCnUXk0Zyq5foU-vinoYbsmFDOX','1yqPThoPIZqyx7Iy42Lc7skWrsGV1p21a',
              '1HeIubKJoiM4R8OFwaQB_Uac1YuWRJ0tK','1yQ4R6fyUoSU5BLhrmLB86fTNf9aa5GGo','1HoSq4RCAfNTpGOuugUQYplvs9b9uAH5D']

for i in range(len(names)):
  name = names[i]; gdrive_id = gdrive_ids[i]
  url = f'https://drive.google.com/uc?id={gdrive_id}'
  output = f'sample_audio_files/{name}'
  gdown.download(url, output, quiet=True)


#Choose a sound for the speech recognizer

In [0]:
audio_file = 'didgeridoo.wav' #@param ['bells.mp3', 'birds.mp3', 'shaker.mp3', 'talkbox.mp3', 'arp_1.wav','car-horn.wav', 'drops.wav', 'skrillex.wav', 'didgeridoo.wav']
audio_file = 'sample_audio_files/'+audio_file


# Or upload your own sound

In [0]:
audio_file = list(files.upload().keys())[0]

# Load the file. Optionally, you can change a few settings for the speech recognizer

### "Offset" refers to where in the file to start from. This can have a big effect, if it changes the recognizer from starting on a consonant to a vowel, for example.

In [132]:
hop_length = 1.5 #@param {type:"slider", min:0.3, max:3.0, step:0.1}
window_length = 1.5 #@param {type:"slider", min:0.3, max:3.0, step:0.1}
# offset time from beginning of the file
offset = 0.1  #@param {type:"slider", min:0.1, max:20, step:0.1}

y, sr = librosa.load(audio_file)
duration_cropped = np.minimum(duration, len(y)/sr - offset)
Audio(y[int(offset*sr):int(duration*sr)], rate=sr)

#Listen to the result after processing the audio with the Vocal Separation algorithm

In [133]:
Audio(separate_vox(y,sr)[0],rate=sr)

# Try generating lyrics. Re-run as many times as you'd like.

In [134]:
words = imagine_words(audio_file, phoneme_model, word_model,
                     phoneme_output_vocab, reverse_phoneme_output_vocab,
                     word_output_vocab, reverse_word_output_vocab,
                     do_separate_vox=True, duration=duration_cropped, tfm=None,
                     hop_length=hop_length, window_length=window_length, offset=offset,
                     dedup_outputs=True, tmpfile_path = 'tmp_file.wav',
                     num_versions=10, num_tries=5)
words = [' '.join(w) for w in words]
words

['et toward et leur',
 'alert leute alert nocturnal and diana',
 'infernal nocturnal infernal carolina infernal',
 'universal nocturnal alert purple corporate',
 'zora all core port door more four more corn',
 'strait it crab crib cut right',
 'corporate out costs trek core for color for',
 'for ciao retro for',
 'for volume you me',
 'union unusual human cuba union an and']